In [1]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

In [4]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks = True)
cap=cv2.VideoCapture(0)

In [6]:
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
LEFT_IRIS = 468
RIGHT_IRIS = 473

In [7]:
screen_w, screen_h = pyautogui.size()
last_click_time = 0
fixation_start_time = None
last_gaze_pos = None
fixation_threshold = 0.03  
fixation_duration = 1.2
zoom_trigger_time = 2.5

In [8]:
def eye_aspect_ratio(landmarks, eye_points):
    p1=landmarks[eye_points[1]]
    p2=landmarks[eye_points[2]]
    p3=landmarks[eye_points[3]]
    p4=landmarks[eye_points[4]]

    vertical = ((p2.y - p4.y) ** 2 + (p2.x - p4.x) ** 2) ** 0.5
    horizontal = ((p1.y - p3.y) ** 2 + (p1.x - p3.x) ** 2) ** 0.5

    return vertical/horizontal

In [9]:
def is_fixated(current_pos, previous_pos, threshold=fixation_threshold):
    if previous_pos is None:
        return False
    dist = np.linalg.norm(np.array(current_pos) - np.array(previous_pos))
    return dist < threshold

In [ ]:
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        mesh = results.multi_face_landmarks[0]
        h, w, _ = frame.shape

        # مختصات مردمک‌ها
        left_iris = mesh.landmark[LEFT_IRIS]
        right_iris = mesh.landmark[RIGHT_IRIS]
        qqqq
        gaze_x = (left_iris.x + right_iris.x) / 2
        gaze_y = (left_iris.y + right_iris.y) / 2

        # تشخیص نگاه ثابت (fixation)
        if is_fixated((gaze_x, gaze_y), last_gaze_pos):
            if fixation_start_time is None:
                fixation_start_time = time.time()
            elapsed = time.time() - fixation_start_time

            # حرکت ماوس فقط در صورت نگاه ثابت
            if elapsed > fixation_duration:
                screen_x = int(gaze_x * screen_w)
                screen_y = int(gaze_y * screen_h)
                pyautogui.moveTo(screen_x, screen_y, duration=0.1)

                # اگر به مرکز نگاه کند و زیاد بماند، زوم کند
                center_x = 0.5
                center_y = 0.5
                if abs(gaze_x - center_x) < 0.05 and abs(gaze_y - center_y) < 0.05:
                    if elapsed > zoom_trigger_time:
                        pyautogui.scroll(500)  # اسکرول به بالا (زوم)
                        time.sleep(1)
        else:
            fixation_start_time = None

        last_gaze_pos = (gaze_x, gaze_y)

        # تشخیص پلک زدن
        left_ear = eye_aspect_ratio(mesh.landmark, LEFT_EYE)
        right_ear = eye_aspect_ratio(mesh.landmark, RIGHT_EYE)

        blink_thresh = 0.22
        left_closed = left_ear < blink_thresh
        right_closed = right_ear < blink_thresh

        if left_closed and right_closed:
            now = time.time()
            if now - last_click_time > 1:
                pyautogui.doubleClick()
                last_click_time = now
            time.sleep(0.6)

        elif left_closed and not right_closed:
            pyautogui.click(button='left')
            time.sleep(0.5)

        elif right_closed and not left_closed:
            pyautogui.click(button='right')
            time.sleep(0.5)

    cv2.imshow("eye_control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 